# Recipe Recommendation System: EDA and Visualization

During preprocessing, we thoroughly cleaned the dataset to ensure it was accurate and consistent. This included fixing data types, removing outliers, and adding new features from existing data. For example, we split nutritional values into separate components, extracted detailed date information, and analyzed dietary preferences. With a clean and organized dataset ready, we can now move on to exploratory data analysis (EDA) to uncover valuable patterns and insights for improving our recipe recommendation system.

In [3]:
# import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
# load the dataset
recipe_df = pd.read_pickle("C:/Users/pd006/Desktop/internship_search/machine_learning/Recipe-Recommender-System/data/recipe.pkl")

In [5]:
dietary_restrictions = ["lactose_intolerance", "gluten_intolerance", "vegetarianism", "veganism", "kosher", "dairy-free", "low-carb"]

In [6]:
restriction_counts = recipe_df[dietary_restrictions].sum()

In [7]:
restriction_counts

lactose_intolerance    3647
gluten_intolerance     3647
vegetarianism          3647
veganism               3647
kosher                 3647
dairy-free             3647
low-carb               3647
dtype: int64